In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bachi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bachi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data_folder = r'C:\Users\bachi\Desktop\IR\Data\01 Legal Case Corpus\cases-by-categories\raw_cases[cases_39155]\raw_cases[cases_39155]'
df = pd.DataFrame({'Case_description': [], 'Case_Type': []})

def read_case_file(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            lines.append(line.strip())
    return lines

def map_casetypes(folder_num):
    new_dataframe = pd.DataFrame(columns=['Case_description', 'Case_Type'])
    path = r'C:\Users\bachi\Desktop\IR\Data\01 Legal Case Corpus\cases-by-categories\raw_cases[cases_39155]\raw_cases[cases_39155]'
    #print(folder_num)
    folder_path = os.path.join(path, str(folder_num))
    if os.path.exists(folder_path):
        for file_name in os.listdir(folder_path):
            #print(file_name)
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path) and file_name.startswith('case') and file_name.endswith('.txt'):
                descriptions = read_case_file(file_path)
                case_description = ' '.join(descriptions)  # Assuming you want to concatenate lines into a single string
                new_row = pd.Series({'Case_description': case_description, 'Case_Type': folder_num})
                new_dataframe = pd.concat([new_dataframe,pd.DataFrame([new_row])], ignore_index=True)
    return new_dataframe

with ThreadPoolExecutor() as executor:
    results = [result for result in executor.map(map_casetypes, range(1, 80)) if result is not None]

# Concatenate the results into a single DataFrame
data = pd.concat(results, ignore_index=True)


In [3]:
data

,Case_description,Case_Type
0,Petitioner Jhonatan Acosta (“Acosta”) petition...,1
1,OPINION Anthony Araujo filed a complaint in th...,1
2,OPINION Robert Zimmerman was riding his motorc...,1
3,OPINION This case returns to us after the Supr...,1
4,"OPINION OF THE COURT Haddrick Byrd, a prisoner...",1
...,...,...
39150,In this claim for benefits under the Black Lun...,78
39151,OPINION James Kea petitions for review of the ...,78
39152,"Mining Energy, Inc., and its insurer, Old Repu...",78
39153,"CSX Hotels, Incorporated, d/b/a The Greenbrier...",78


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39155 entries, 0 to 39154
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Case_description  39155 non-null  object
 1   Case_Type         39155 non-null  object
dtypes: object(2)
memory usage: 611.9+ KB


In [5]:
data.tail(10)

,Case_description,Case_Type
39145,"Valeria Spencer (“Spencer”), the plaintiff-bel...",78
39146,"Appellant, Foster Rawley, is an injured worker...",78
39147,"Harold Munyan (“Munyan”), employee-appellant, ...",78
39148,Claimant Michael Falconi appeals from a judgme...,78
39149,OPINION Benjamin S. Reynolds (“Appellant”) app...,78
39150,In this claim for benefits under the Black Lun...,78
39151,OPINION James Kea petitions for review of the ...,78
39152,"Mining Energy, Inc., and its insurer, Old Repu...",78
39153,"CSX Hotels, Incorporated, d/b/a The Greenbrier...",78
39154,OPINION On James Martin's application for blac...,78


In [6]:
data['Case_Type'] = data['Case_Type'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39155 entries, 0 to 39154
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Case_description  39155 non-null  object
 1   Case_Type         39155 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 459.0+ KB


In [7]:
nltk.download('omw-1.4')
nltk.download('punkt')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bachi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bachi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:

def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Ensure each token is alphanumeric and not in stop words
    words = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]
    
    # Lemmatize each word
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

data['processed_description'] = data['Case_description'].apply(preprocess_text)
data

,Case_description,Case_Type,processed_description
0,Petitioner Jhonatan Acosta (“Acosta”) petition...,1,petitioner jhonatan acosta acosta petition cou...
1,OPINION Anthony Araujo filed a complaint in th...,1,opinion anthony araujo filed complaint united ...
2,OPINION Robert Zimmerman was riding his motorc...,1,opinion robert zimmerman riding motorcycle sum...
3,OPINION This case returns to us after the Supr...,1,opinion case return u supreme court review rey...
4,"OPINION OF THE COURT Haddrick Byrd, a prisoner...",1,opinion court haddrick byrd prisoner filed pro...
...,...,...,...
39150,In this claim for benefits under the Black Lun...,78,claim benefit black lung benefit act act 30 we...
39151,OPINION James Kea petitions for review of the ...,78,opinion james kea petition review decision ord...
39152,"Mining Energy, Inc., and its insurer, Old Repu...",78,mining energy insurer old republic insurance c...
39153,"CSX Hotels, Incorporated, d/b/a The Greenbrier...",78,csx hotel incorporated greenbrier hotel greenb...


In [10]:
file_path = r"C:\Users\bachi\Desktop\IR\Data\01 Legal Case Corpus\cases-by-categories\Map.txt"

column_names = ['Case_Type', 'Case_Type_str']

# Load data into a DataFrame
class_names = pd.read_csv(file_path, delimiter='->', names = column_names)

# Display the DataFrame
print(class_names)

C:\Users\bachi\AppData\Local\Temp\ipykernel_14896\2329356449.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  class_names = pd.read_csv(file_path, delimiter='->', names = column_names)


    Case_Type                      Case_Type_str
0           1                 Administrative Law
1           2                          Admiralty
2           3                        Agriculture
3           4   Antitrust &amp; Trade Regulation
4           5                    Attorney's Fees
..        ...                                ...
71         74           Tax-exempt Organizations
72         75                     Transportation
73         76                          Water Law
74         77                 White Collar Crime
75         78              Workers' Compensation

[76 rows x 2 columns]


In [11]:
merged_df = pd.merge(data, class_names, on='Case_Type')
preprocessed_df = merged_df.drop('Case_description', axis=1)
preprocessed_df = preprocessed_df.drop('Case_Type', axis=1)
preprocessed_df

,processed_description,Case_Type_str
0,petitioner jhonatan acosta acosta petition cou...,Administrative Law
1,opinion anthony araujo filed complaint united ...,Administrative Law
2,opinion robert zimmerman riding motorcycle sum...,Administrative Law
3,opinion case return u supreme court review rey...,Administrative Law
4,opinion court haddrick byrd prisoner filed pro...,Administrative Law
...,...,...
39150,claim benefit black lung benefit act act 30 we...,Workers' Compensation
39151,opinion james kea petition review decision ord...,Workers' Compensation
39152,mining energy insurer old republic insurance c...,Workers' Compensation
39153,csx hotel incorporated greenbrier hotel greenb...,Workers' Compensation


In [12]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_df['processed_description'])
X = tokenizer.texts_to_sequences(preprocessed_df['processed_description'])
print(X[:1])
y = preprocessed_df['Case_Type_str']

[[196, 181413, 7521, 7521, 116, 1, 41, 32, 102, 673, 28, 381, 2345, 673, 98, 467, 32, 7521, 3997, 805, 535, 188, 21, 2, 72, 809, 10539, 673, 3508, 24, 2143, 1827, 208, 176, 3467, 208, 352, 381, 467, 463, 162, 142, 467, 2290, 486, 867, 381, 463, 4744, 2345, 467, 32, 361, 1247, 1035, 8857, 278, 111, 1108, 1090, 116, 320, 485, 651, 2784, 650, 9297, 7521, 89, 1428, 2390, 7451, 3262, 1444, 3311, 21, 2, 650, 11179, 45, 112, 432, 946, 7521, 309, 2550, 1051, 1399, 5420, 465, 21, 2, 2629, 673, 48, 7296, 91, 99, 556, 13, 1499, 809, 21, 2, 7130, 1164, 555, 896, 283, 6341, 89, 1428, 13, 487, 2274, 266, 555, 1121, 7296, 154, 99, 159, 2274, 266, 1119, 149, 7521, 942, 416, 84, 95, 1292, 114, 981, 9816, 247, 5481, 272, 3647, 535, 188, 21, 2, 72, 809, 467, 7521, 524, 184, 206, 256, 12886, 621, 809, 21, 2, 150, 381, 1240, 809, 447, 721, 535, 3180, 188, 2997, 90, 2199, 558, 4, 2629, 56, 443, 6199, 117, 533, 49, 673, 30, 73, 113882, 647, 3573, 4220, 381, 946, 6158, 1564, 1060, 40, 126, 955, 283, 7521, 867

In [13]:
# Padding sequences
max_length = 200
padded_X = pad_sequences(X, maxlen=max_length, padding='post', truncating='post')

In [19]:
from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format(r"C:\Users\bachi\Desktop\IR\Data\02 Legal domain word2vec models\raw-legal\raw-legal\no replacements\legalrawwoldmodel.bin", binary=True, unicode_errors='ignore')

In [20]:
def create_embedding_matrix(word2vec_model, tokenizer, embedding_dim):
    vocab_size = len(tokenizer.word_index) + 1  # Adding 1 to account for the padding token
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if word in word2vec_model:
            embedding_matrix[i] = word2vec_model[word]

    return embedding_matrix

embedding_dim = word2vec_model.vector_size
embedding_matrix = create_embedding_matrix(word2vec_model, tokenizer, embedding_dim)

In [21]:
import pickle

# Save padded_sequences and embedding_matrix to disk
np.save(r'C:\Users\bachi\Desktop\IR\Code\padded_X.npy', padded_X)
with open(r'C:\Users\bachi\Desktop\IR\Code\embedding_matrix.pkl', 'wb') as f:
    pickle.dump(embedding_matrix, f)

In [18]:
y.to_csv(r'C:\Users\bachi\Desktop\IR\Code\y.csv', index=False)

In [22]:
data.to_csv(r'C:\Users\bachi\Desktop\IR\Code\data.csv', index=True)